**Código principal 2º base de datos**

Filtro de Madgwick sin magnetómetro

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from madgwickahrs import MadgwickAHRS
from quaternion import Quaternion

def load_csv_data(csv_file_path):
    data = pd.read_csv(csv_file_path)
    data.columns = data.columns.str.strip()  # Remove leading and trailing spaces from column names
    acc = data[['Acc_X', 'Acc_Y', 'Acc_Z']].values
    gyr = data[['Gyr_X', 'Gyr_Y', 'Gyr_Z']].values
    return acc, gyr

def calculate_euler_angles(acc, gyr):
    madgwick = MadgwickAHRS()
    euler_angles = []

    for a, g in zip(acc, gyr):
        madgwick.update_imu(g, a)
        quat = madgwick.quaternion
        euler = Quaternion.to_euler_angles(quat)
        euler_angles.append(euler)

    return np.array(euler_angles)

def plot_euler_angles(euler_angles):
    time = np.arange(euler_angles.shape[0])
    plt.figure(figsize=(10, 6))
    plt.plot(time, euler_angles[:, 0], label='Roll')
    plt.plot(time, euler_angles[:, 1], label='Pitch')
    plt.plot(time, euler_angles[:, 2], label='Yaw')
    plt.xlabel('Time')
    plt.ylabel('Angle (radians)')
    plt.title('Euler Angles over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

def main():
    csv_file_path = './BD 1/input data_SD/5/5-000_00B432B6.txt.csv'
    acc, gyr = load_csv_data(csv_file_path)
    
    euler_angles = calculate_euler_angles(acc, gyr)
    
    print("Calculated Euler Angles:")
    print(euler_angles)
    
    plot_euler_angles(euler_angles)

if __name__ == "__main__":
    main()


Filtro de Madgwick con magnetómetro

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from madgwickahrs import MadgwickAHRS
from quaternion import Quaternion

def load_csv_data(csv_file_path):
    data = pd.read_csv(csv_file_path)
    data.columns = data.columns.str.strip()  # Remove leading and trailing spaces from column names
    acc = data[['Acc_X', 'Acc_Y', 'Acc_Z']].values
    gyr = data[['Gyr_X', 'Gyr_Y', 'Gyr_Z']].values
    mag = data[['Mag_X', 'Mag_Y', 'Mag_Z']].values
    euler_csv = data[['Yaw', 'Roll', 'Pitch']].values  # Load Euler angles from CSV
    print(data.keys())
    return acc, gyr, mag, euler_csv

def calculate_euler_angles(acc, gyr, mag):
    madgwick = MadgwickAHRS()
    euler_angles = []

    for a, g, m in zip(acc, gyr, mag):
        madgwick.update(g, a, m)
        quat = madgwick.quaternion
        euler = Quaternion.to_euler_angles(quat)
        euler_angles.append(euler)

    euler_angles = np.array(euler_angles)
    euler_angles_unwrapped = np.unwrap(euler_angles, axis=0)  # Apply unwrap to eliminate jumps
    euler_angles_deg = np.degrees(euler_angles_unwrapped)  # Convert to degrees
    return euler_angles_deg

def unwrap_angles(euler_angles):
    euler_angles_unwrapped = np.unwrap(np.radians(euler_angles), axis=0)  # Convert to radians and apply unwrap
    return np.degrees(euler_angles_unwrapped)  # Convert back to degrees

def plot_euler_angles_madgwick(euler_angles_madgwick):
    time = np.arange(euler_angles_madgwick.shape[0]) / 100.0  # Assuming 120 Hz sampling rate
    plt.figure(figsize=(10, 6))
    plt.plot(time, euler_angles_madgwick[:, 0], label='Roll (Madgwick)', color='orange')
    plt.plot(time, euler_angles_madgwick[:, 1], label='Pitch (Madgwick)', color='green')
    plt.plot(time, euler_angles_madgwick[:, 2], label='Yaw (Madgwick)', color='blue')
    plt.xlabel('Time (s)')
    plt.ylabel('Angle (degrees)')
    plt.title('Euler Angles from Madgwick over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_euler_angles_csv(euler_angles_csv):
    time = np.arange(euler_angles_csv.shape[0]) / 100.0  # Assuming 120 Hz sampling rate
    plt.figure(figsize=(10, 6))
    plt.plot(time, euler_angles_csv[:, 1], label='Roll (CSV)', color='orange', linestyle='dashed')
    plt.plot(time, euler_angles_csv[:, 2], label='Pitch (CSV)', color='green', linestyle='dashed')
    plt.plot(time, euler_angles_csv[:, 0], label='Yaw (CSV)', color='blue', linestyle='dashed')
    plt.xlabel('Time (s)')
    plt.ylabel('Angle (degrees)')
    plt.title('Euler Angles from CSV over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

csv_file_path = './BD 1/input data_SD/1/5-000_00B43293.txt.csv'
acc, gyr, mag, euler_csv = load_csv_data(csv_file_path)

euler_angles_madgwick = calculate_euler_angles(acc, gyr, mag)

print("Calculated Euler Angles (in degrees) from Madgwick:")
print(euler_angles_madgwick)

print("Euler Angles from CSV (in degrees):")
print(euler_csv)

plot_euler_angles_madgwick(euler_angles_madgwick)
plot_euler_angles_csv(euler_csv)

csv_file_path = './Bases_combinadas/5-000_00B432B6_comb.csv'
acc, gyr, mag, euler_csv = load_csv_data(csv_file_path)

euler_angles_madgwick = calculate_euler_angles(acc, gyr, mag)

print("Calculated Euler Angles (in degrees) from Madgwick:")
print(euler_angles_madgwick)

print("Euler Angles from CSV (in degrees):")
print(euler_csv)

plot_euler_angles_madgwick(euler_angles_madgwick)
plot_euler_angles_csv(euler_csv)
